### TODO:
1. ellenőrizni megyéket: lett évről-évre eltérés? --> ha nincs nagy eltérés, hozzuk egy oszlopra
2. adatokat aggregálni megye illetve gazdálkodási forma és ágazat kategória szerint
3. feature_importance check
4. cross-validation
5. number of trees to use: plot the training: num trees on x and error on y axis

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk

In [47]:
from sklearn.model_selection import train_test_split

dataset_path = './dataset'
target_column = 'target_reg'

train_df = pd.read_csv(f'{dataset_path}/train.csv')

# Clean data, reduce features

In [52]:
train_df.dropna(inplace=True)

from the 3 megye columns 1 is enough, since there was no change in any of the rows

In [50]:
train_df_reduced_features = train_df.drop(['MEGYE_2015', 'MEGYE_2016'], axis='columns')
train_df_reduced_features.rename({'MEGYE_2014': 'megye'}, inplace=True)

### target_reg min miért negatív? létezik negatív export? + plottal meg lehetne vizsgálni

In [5]:
print(train_df.describe())

                 id    target_reg  KATEGORIKUS_VALTOZO_2014  \
count  31745.000000  3.174500e+04              31745.000000   
mean   16388.415404  2.469644e+05                  0.081997   
std     9573.589176  2.466065e+06                  0.274365   
min        1.000000 -1.650300e+04                  0.000000   
25%     8139.000000  0.000000e+00                  0.000000   
50%    16196.000000  0.000000e+00                  0.000000   
75%    24672.000000  7.800000e+01                  0.000000   
max    33180.000000  1.047418e+08                  1.000000   

       KATEGORIKUS_VALTOZO_2015  KATEGORIKUS_VALTOZO_2016   aktelh_2014  \
count              31745.000000              31745.000000  3.174500e+04   
mean                   0.079666                  0.078343  1.644770e+04   
std                    0.270780                  0.268715  2.977193e+05   
min                    0.000000                  0.000000  0.000000e+00   
25%                    0.000000                  0.000000

In [3]:
from sklearn.preprocessing import OneHotEncoder
from utils import columns_to_onehotencode

encoder = OneHotEncoder(handle_unknown='ignore')

encoder_df = pd.DataFrame(encoder.fit_transform(train_df[[*columns_to_onehotencode]]).toarray())
train_df.reset_index(inplace=True)
encoder_df.reset_index(inplace=True)
train_df = pd.concat([train_df, encoder_df], axis='columns')
print(train_df.shape)
print(train_df.isna().sum().sum())

(31745, 6704)
0


# Train

In [4]:
target_df = train_df[target_column]
train_df.drop([target_column, *columns_to_onehotencode], axis='columns', inplace=True)

train_df.columns = train_df.columns.map(str)

x_train, x_test, y_train, y_test = train_test_split(train_df, target_df, random_state=42)

In [5]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=50, random_state=42)
rf.fit(x_train,y_train)
predictions = rf.predict(x_test)

mean squared error: 287095634850.57904


# Evaluate

In [6]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
mse = mean_squared_error(predictions, y_test)
mae = mean_absolute_error(predictions, y_test)
print(f'mean squared error: {mse}')
print(f'mean absolute error: {mae}')

mean squared error: 287095634850.57904
mean absolute error: 56776.8791306539
